## 1. Import libraries

In [2]:
import sys
sys.path.append('../')

%matplotlib inline
from collections import defaultdict
from typing import List
from sklearn.preprocessing import StandardScaler

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots
import src.serializer as serializer

## 2. Auxiliary functions

### 2.1. Reduce state space

In [3]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob

### 2.2. Reward policy

In [4]:
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

### 2.3. Print results

In [5]:
def print_result(i, scores, total_reward, score):
#     if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

## 3. Running algorithms

### 3.1. Hyperparameters

In [6]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

GAMMA = 0.99
AVAILABLE_ACTIONS = 2
N0 = 2.5
ALPHA = 0.00001

reward_policy.REWARD_IF_CROSS = 50
reward_policy.REWARD_IF_COLISION = -1
reward_policy.REWARD_IF_STILL = -0.1

### 3.2. Monte Carlo

In [ ]:
env, initial_state = environment.get_env()

mc_agent = agents.MonteCarloControl(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
def MonteCarloES(agent, reduce_s, reward_p, RAM_mask: List[int], render: bool=False):
    epi = episode.generate_episode(env, reduce_state=reduce_s, reward_policy=reward_p, agent=agent, RAM_mask=RAM_mask, render=render)
    return agent.update_policy(epi)

In [ ]:
# %%time
# MonteCarloES(agent=mc_agent,reduce_s=reduce_state, reward_p=reward_policy, RAM_mask=RAM_mask, render=False)

In [ ]:
%%time
scores = []
total_rewards = []


n_runs = 5

for i in range(n_runs):
    render = i % 201 == 200

    score, total_reward = MonteCarloES(agent=mc_agent,reduce_s=reduce_state, reward_p=reward_policy, RAM_mask=RAM_mask, render=render)

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

### 3.3. Q Learning

In [ ]:
env, initial_state = environment.get_env()
ql_agent = agents.QLearning(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
%%time
scores = []
total_rewards = []

n_runs = 1
render = False
for i in range(n_runs):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        ql_agent.update_Q(old_state, state, action, reward)

        action = ql_agent.act(state)  # Next action

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

In [ ]:
for key in ql_agent.Q:
    print('{}:{}'.format(np.frombuffer(key, dtype=np.uint8, count=-1), ql_agent.Q[key]))

### 3.4. Q Learning Approximator

In [7]:
env, initial_state = environment.get_env()
ql_agent_app = agents.QLearningLinearApprox(alpha=ALPHA, gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0, weights_length=4) #len(RAM_mask)
ql_agent_app.trainScaler(env, RAM_mask, feat_type='mean')

In [8]:
state = env.reset()
ql_agent_app.createFeature(reduce_state(state)[RAM_mask].data.tobytes(), 1,feat_type='mean')

array([0.02704611, 0.        , 0.        , 0.        , 1.        ,
       1.        ])

In [10]:
%%time
scores = []
total_rewards = []
feat_type = 'mean'

n_runs = 10
render = False
for i in range(n_runs):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent_app.act(state, feat_type=feat_type)
    
    score = 0
    total_reward = 0

    count = 0
    print("Episode ",i)
    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()
        print("Run ", count)
        ql_agent_app.update_W(old_state, state, action, reward, fixed_alpha=True, feat_type=feat_type)

        action = ql_agent_app.act(state,feat_type=feat_type)  # Next action
        count+=1
        print('------------------------------------')
    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)
    

Episode  0
Run  0
------------------------------------
Run  1
------------------------------------
Run  2
------------------------------------
Run  3
------------------------------------
Run  4
------------------------------------
Run  5
------------------------------------
Run  6
------------------------------------
Run  7
------------------------------------
Run  8
------------------------------------
Run  9
------------------------------------
Run  10
------------------------------------
Run  11
------------------------------------
Run  12
------------------------------------
Run  13
------------------------------------
Run  14
------------------------------------
Run  15
------------------------------------
Run  16
------------------------------------
Run  17
------------------------------------
Run  18
------------------------------------
Run  19
------------------------------------
Run  20
------------------------------------
Run  21
------------------------------------
Run  22
-

------------------------------------
Run  233
------------------------------------
Run  234
------------------------------------
Run  235
------------------------------------
Run  236
------------------------------------
Run  237
------------------------------------
Run  238
------------------------------------
Run  239
------------------------------------
Run  240
------------------------------------
Run  241
------------------------------------
Run  242
------------------------------------
Run  243
------------------------------------
Run  244
------------------------------------
Run  245
------------------------------------
Run  246
------------------------------------
Run  247
------------------------------------
Run  248
------------------------------------
Run  249
------------------------------------
Run  250
------------------------------------
Run  251
------------------------------------
Run  252
------------------------------------
Run  253
----------------------------------

------------------------------------
Run  480
------------------------------------
Run  481
------------------------------------
Run  482
------------------------------------
Run  483
------------------------------------
Run  484
------------------------------------
Run  485
------------------------------------
Run  486
------------------------------------
Run  487
------------------------------------
Run  488
------------------------------------
Run  489
------------------------------------
Run  490
------------------------------------
Run  491
------------------------------------
Run  492
------------------------------------
Run  493
------------------------------------
Run  494
------------------------------------
Run  495
------------------------------------
Run  496
------------------------------------
Run  497
------------------------------------
Run  498
------------------------------------
Run  499
------------------------------------
Run  500
----------------------------------

Run  718
------------------------------------
Run  719
------------------------------------
Run  720
------------------------------------
Run  721
------------------------------------
Run  722
------------------------------------
Run  723
------------------------------------
Run  724
------------------------------------
Run  725
------------------------------------
Run  726
------------------------------------
Run  727
------------------------------------
Run  728
------------------------------------
Run  729
------------------------------------
Run  730
------------------------------------
Run  731
------------------------------------
Run  732
------------------------------------
Run  733
------------------------------------
Run  734
------------------------------------
Run  735
------------------------------------
Run  736
------------------------------------
Run  737
------------------------------------
Run  738
------------------------------------
Run  739
-------------------------

------------------------------------
Run  974
------------------------------------
Run  975
------------------------------------
Run  976
------------------------------------
Run  977
------------------------------------
Run  978
------------------------------------
Run  979
------------------------------------
Run  980
------------------------------------
Run  981
------------------------------------
Run  982
------------------------------------
Run  983
------------------------------------
Run  984
------------------------------------
Run  985
------------------------------------
Run  986
------------------------------------
Run  987
------------------------------------
Run  988
------------------------------------
Run  989
------------------------------------
Run  990
------------------------------------
Run  991
------------------------------------
Run  992
------------------------------------
Run  993
------------------------------------
Run  994
----------------------------------

...

Run  1709
------------------------------------
Run  1710
------------------------------------
Run  1711
------------------------------------
Run  1712
------------------------------------
Run  1713
------------------------------------
Run  1714
------------------------------------
Run  1715
------------------------------------
Run  1716
------------------------------------
Run  1717
------------------------------------
Run  1718
------------------------------------
Run  1719
------------------------------------
Run  1720
------------------------------------
Run  1721
------------------------------------
Run  1722
------------------------------------
Run  1723
------------------------------------
Run  1724
------------------------------------
Run  1725
------------------------------------
Run  1726
------------------------------------
Run  1727
------------------------------------
Run  1728
------------------------------------
Run  1729
------------------------------------
Run  1730
---

------------------------------------
Run  1963
------------------------------------
Run  1964
------------------------------------
Run  1965
------------------------------------
Run  1966
------------------------------------
Run  1967
------------------------------------
Run  1968
------------------------------------
Run  1969
------------------------------------
Run  1970
------------------------------------
Run  1971
------------------------------------
Run  1972
------------------------------------
Run  1973
------------------------------------
Run  1974
------------------------------------
Run  1975
------------------------------------
Run  1976
------------------------------------
Run  1977
------------------------------------
Run  1978
------------------------------------
Run  1979
------------------------------------
Run  1980
------------------------------------
Run  1981
------------------------------------
Run  1982
------------------------------------
Run  1983
-------------

------------------------------------
Run  2200
------------------------------------
Run  2201
------------------------------------
Run  2202
------------------------------------
Run  2203
------------------------------------
Run  2204
------------------------------------
Run  2205
------------------------------------
Run  2206
------------------------------------
Run  2207
------------------------------------
Run  2208
------------------------------------
Run  2209
------------------------------------
Run  2210
------------------------------------
Run  2211
------------------------------------
Run  2212
------------------------------------
Run  2213
------------------------------------
Run  2214
------------------------------------
Run  2215
------------------------------------
Run  2216
------------------------------------
Run  2217
------------------------------------
Run  2218
------------------------------------
Run  2219
------------------------------------
Run  2220
-------------

Run  2408
------------------------------------
Run  2409
------------------------------------
Run  2410
------------------------------------
Run  2411
------------------------------------
Run  2412
------------------------------------
Run  2413
------------------------------------
Run  2414
------------------------------------
Run  2415
------------------------------------
Run  2416
------------------------------------
Run  2417
------------------------------------
Run  2418
------------------------------------
Run  2419
------------------------------------
Run  2420
------------------------------------
Run  2421
------------------------------------
Run  2422
------------------------------------
Run  2423
------------------------------------
Run  2424
------------------------------------
Run  2425
------------------------------------
Run  2426
------------------------------------
Run  2427
------------------------------------
Run  2428
------------------------------------
Run  2429
---

Run  2593
------------------------------------
Run  2594
------------------------------------
Run  2595
------------------------------------
Run  2596
------------------------------------
Run  2597
------------------------------------
Run  2598
------------------------------------
Run  2599
------------------------------------
Run  2600
------------------------------------
Run  2601
------------------------------------
Run  2602
------------------------------------
Run  2603
------------------------------------
Run  2604
------------------------------------
Run  2605
------------------------------------
Run  2606
------------------------------------
Run  2607
------------------------------------
Run  2608
------------------------------------
Run  2609
------------------------------------
Run  2610
------------------------------------
Run  2611
------------------------------------
Run  2612
------------------------------------
Run  2613
------------------------------------
Run  2614
---

In [12]:
exp = serializer.Experiment(ql_agent_app, scores, total_rewards, reduce_state, reward_policy)
exp.save_experiment('.','QL')

In [13]:
exp.load_experiment('.QL_2020_12_24_02_54_49.dill')

In [ ]:
with open("scores.txt", "w") as f:
    for s,tr in zip(scores, total_rewards):
        f.write("{}, {}\n".format(s, tr))

In [ ]:
aux_plots.plot_scores(scores)

In [ ]:
aux_plots.plot_rewards(total_rewards)

In [ ]:
ql_agent_app.update_W(old_state, state, action, reward)

In [ ]:
old_state, state, action, reward

In [ ]:
teste = np.frombuffer(state, dtype=np.uint8, count=-1)
teste.reshape(1,-1)

In [ ]:
scaler = StandardScaler(with_mean=True)
# observation_examples = np.array([env.observation_space.sample()[RAM_mask] for x in range(10000)])
# scaler.fit(observation_examples)

In [ ]:
scaler.transform(a.reshape(1,-1))

In [ ]:
state = observation_examples[0]
state1=state[0:2]
state2=np.mean(state[2:])
state3=np.count_nonzero(state[2:])
a=np.array(1)
state=np.concatenate((state1,state2,state3,a), axis=None)
a = np.concatenate((state[0:2], np.mean(state[2:]), np.count_nonzero(state[2:])), axis=None)

In [ ]:
observations = [env.observation_space.sample()[RAM_mask] for x in range(10000)]
features = [np.concatenate((state[0:2], np.mean(state[2:]), np.count_nonzero(state[2:])), axis=None)
for state in observations]

In [ ]:
scaler.fit(np.array(features))

In [ ]:
np.array(features)